In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random

Graph $(V, E)$ with nodes $v\subset\mathbb{N}$ as subsets of integers
Strategy for merging:
1. Draw a set of edges $F\subseteq E$ somehow
2. Identify connected components $\mathcal{C}=\{C_1,\dotsc, C_k\}$ of $(V, F)$. They form the new set of nodes
3. Compute new adjacency matrix $A^\prime$ by summing entries of $A$ blockwise 

#### Rule of thumb for choosing temperature $\tau$:
If there are $m$ edges in an unweighted graph, the amount of chosen edges is distrubted as Bin($e$, $n^{-\tau}$). Therefore, the expected number of chosen edges is $n^{1-\tau}$, and a choice of 
$$
\tau=\frac{\log(2)}{\log(m)}
$$
leads to $m/2$ in expectation.

In [379]:
class FindCuts(object):
    def __init__(self, A, sample_fn, partition_fn, quality_fn):
        self.A = A
        self.V = [{i} for i in range(nx.number_of_nodes(G))]
        self.sample_fn = sample_fn
        self.partition_fn = partition_fn
        self.quality_fn = quality_fn
    
    def _merge_nodes(self, V, component):
        """ Merge the sets of V that are indexed in component."""
        merged_node = set({})
        for idx in component:
            merged_node = merged_node.union(V[idx])
        return merged_node
    
    def _merge_edges(self, V, A, A_sampled):
        """ Merge edges in A_sampled to obtain a new graph."""
        G_sampled = nx.from_numpy_array(A_sampled)
        connected_components = list(nx.connected_components(G_sampled))
        V_coarse = [self._merge_nodes(V, component) for component in connected_components]
        n_coarse = len(V_coarse)
        A_coarse = np.zeros((n_coarse, n_coarse))
        for i in range(n_coarse):
            for j in range(i+1, n_coarse):
                for v in connected_components[i]:
                    for w in connected_components[j]:
                        A_coarse[i, j] += A[v, w] 
        A_coarse += A_coarse.T
        return V_coarse, A_coarse
        
    def get_cuts(self, T):
        """ T: number of partitions"""
        V = self.V
        A = self.A
        for t in range(T):
            A_sampled = self.sample_fn(A)
            V, A = self._merge_edges(V, A, A_sampled)
            if len(V)==1:
                return
            P = self.partition_fn(V, A, n=self.A.shape[0])
            print(len(V), self.quality_fn(P), wcut(self.A, np.ones(self.A.shape[0]), P))
        return
    

# Weighted ratio cut with weighted kernelized k-means

__Input:__ Adjacency matrix $A$ with weighted edges, $w$ vector of node weights. <br>
__Output:__ Cluster indicator vector x

Weighted ratio cut (Wcut) is given by
$$
Wcut(A,A^c) = \left(\frac{1}{w(A)}+\frac{1}{w(A^c)}\right)cut(A,A^c)\,,
$$
where
$$
w(A) = \sum_{i\in A}w(i)\quad\text{and}\quad cut(A,A^c)=\sum_{i\in A, j\in A^c}w(i,j)
$$

In [386]:
def wcut(A, w, x):
    """ Computes weighted ratio cut value for a given flat vector x."""
    if w is None:
        w = np.ones(A.shape[0])
    return (1 / (x @ w) + 1 / ((1 - x) @ w)) * (x @ A) @ (1 - x)

def compute_wcut(A, w, sigma=0, t_max=100):
    """ Compute weighted ratio cut with weighted kernel 2-means."""
    # Compute appropriate kernel matrix
    if w is None:
        w = np.ones(A.shape[0])
    
    L = np.diag(np.sum(A, axis=1)) - A
    K = np.diag(1 / w) * sigma + (np.diag(1 / w) @ (np.diag(w) - L)) @ np.diag(1 / w)
    
    # Perform weighted kernelized k-means with random initialization
    n = A.shape[0]
    x_old = np.random.binomial(n=1, p=.5, size=n)
    x_old[0] = 1 - x_old[-1] # Hacky way to avoid having a vector of all 0 or all 1
    D = np.zeros((n, 2))
    for t in range(t_max):
        for b in [0, 1]:
            w_b = w * (x_old==b)
            w_b_sum = w_b.sum()
            D[:, b] = -2 * (K @ w_b) / w_b_sum + np.sum(w_b.reshape(-1, 1) * K * w_b) / w_b_sum ** 2
        x_new = np.argmin(D, axis=1)        
        if (x_old==x_new).all():
            break
        x_old = x_new
    return x_new

def compute_best_wcut(A, w=None, sigmas=np.linspace(-2, 2, 20)):
    """ Compute weighted ratio cut for several sigma and take best."""
    x_best = []
    best_val = np.inf
    for sigma in sigmas:
        x = compute_wcut(A, w, sigma=sigma)
        if wcut(A, w, x) < best_val:
            x_best = x
            best_val = wcut(A, w, x)
    return x_best

def partition(V, A, n):
    """ Compute coarse partition and translate to partition of original graph."""
    x_coarse = compute_spectral_wcut(A=A, w=np.array([len(v) for v in V]))
    x = np.zeros(n)
    for i,v in enumerate(V):
        x[list(v)]= x_coarse[i]
    return x

def evaluate_partition_quality(x, sizes):
    """ Evaluate the fractions that are separated by a partition in an SBM."""
    fractions = []
    pos = 0
    for i in range(len(sizes)):
        fractions.append(x[pos: pos + sizes[i]].mean())
        pos += sizes[i]
    return fractions

### Example: Stochastical Block model

In [387]:
sizes = np.array([500, 500])
G = nx.stochastic_block_model(sizes=sizes, p=[[.8, .1],
                                                 [.1, .8]])
A = nx.to_numpy_array(G)
V = [{i} for i in range(A.shape[0])]

In [388]:
findCuts = FindCuts(A=A, sample_fn=sample_edges, partition_fn=partition, 
                    quality_fn=lambda x: evaluate_partition_quality(x, sizes=sizes))
findCuts.get_cuts(T=30)

867 [0.028, 0.99] 125.2005649830545
737 [0.046, 0.98] 143.71315008946047
631 [0.064, 0.956] 170.4001600640256
523 [0.92, 0.07] 196.49564956495647
435 [0.066, 0.866] 224.3493915866969
368 [0.892, 0.164] 263.3980161787365
320 [0.848, 0.166] 285.89203483882835
271 [0.17, 0.814] 303.7137507201844
223 [0.924, 0.392] 340.09225190636164
189 [0.086, 0.594] 349.3360071301248
162 [0.094, 0.554] 365.1563298999196
134 [0.082, 0.446] 390.03314393939394
116 [0.058, 0.372] 397.9914086801955
106 [0.948, 0.696] 411.2263867246234
97 [0.966, 0.786] 423.57121814700247
87 [0.998, 0.848] 422.43671821136616
75 [0.004, 0.152] 423.45236108793586
67 [0.992, 0.854] 427.1784553474695
58 [0.998, 0.902] 432.77894736842103
48 [0.0, 0.074] 434.9021919115377
41 [0.004, 0.066] 438.7860843819393
34 [0.998, 0.992] 438.99497487437185
29 [0.008, 0.024] 446.39227642276427
21 [0.022, 0.004] 449.1465980827684
14 [0.988, 0.996] 443.67439516129036
13 [0.994, 0.998] 436.49598393574297
11 [0.994, 0.998] 436.49598393574297
9 [0.99

In [8]:
wcut(A=A, w=None, x=compute_best_wcut(A, sigmas=np.linspace(-10,10,100)))

41.666666666666664

# Yu and Shi Postprocessing: spectral approach to Wcut

In [40]:
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh

In [377]:
def compute_spectral_wcut(A, w=None, K=2, max_iter=50):
    """ Solves weighted ratio cut as a relaxed trace maximization problem with Yu and Shi postprocessing."""
    N = A.shape[0]
    if w is None:
        w = np.ones(N)
    L = np.diag(np.sum(A, axis=1)) - A
    
    # Solve eigenvalue problem
    w_inv_sqrt = w**(-1/2)
    s, V = eigsh(np.eye(L.shape[0]) - w_inv_sqrt.reshape(-1, 1) * L * w_inv_sqrt,
                 k=K, which='LA')
    s, V = s[::-1], V[:,::-1]
    Z = w_inv_sqrt.reshape(-1, 1) * V
    assert np.isclose((np.diag(1/w) @ (np.diag(w)-L)) @ Z, Z @ np.diag(s)).all(), 'Eigendecomposition failed'
    
    # Normalize solution
    X_tilde = 1 / np.linalg.norm(Z, axis=1, keepdims=True) * Z
    assert np.isclose(np.linalg.norm(X_tilde, axis=1), np.ones(X_tilde.shape[0])).all(), 'Normalization failed'
    
    # Initialize R
    R = np.zeros((K, K))
    R[:, 0] = X_tilde[random.randrange(N), :]
    c = np.zeros(N)
    for k in range(1, K):
        c += np.abs(X_tilde @ R[:, k-1])
        R[:, k] = X_tilde[np.argmin(c), :]
    
    # Update X and R iteratively until convergence
    psi_old = 0
    for _ in range(max_iter):
        # Update X
        arg_maxs = np.argmax(X_tilde @ R, axis=1)
        X = np.zeros_like(X_tilde)
        X[np.arange(N), arg_maxs] = 1
        
        # Update R
        U, omega, U_tilde = np.linalg.svd(X.T @ X_tilde)
        assert np.isclose((U * omega) @ U_tilde, X.T @ X_tilde).all(), 'SVD failed'
        psi_new = omega.sum()
#         print(np.abs(psi_old-psi_new))
        # Check for convergence
        if np.isclose(psi_new, psi_old):
#             print(f'Converged after {_} steps')
            break
        else:
            psi_old = psi_new
        R = (U @ U_tilde).T
    return X[:, 0]

#### Tests

In [367]:
sizes = np.array([50, 50])
G = nx.stochastic_block_model(sizes=sizes, p=[[.8, .1],
                                                 [.1, .8]])
A = nx.to_numpy_array(G)
V = [{i} for i in range(A.shape[0])]

In [368]:
X = compute_spectral_wcut(A)
x = X[:,0]

99.8011083854664
0.0
Converged after 1 steps


### Evaluation: comparison of different cut's values

In [369]:
spectral_cut = wcut(A=A, w=None, x=x) 
x_blocks = np.zeros(A.shape[0])
x_blocks[:sizes[0]] = 1
block_cut = wcut(A, np.ones(A.shape[0]), x_blocks)

P = np.random.binomial(n=1, p=.5, size=A.shape[0])
random_cut = wcut(A, None, P)
print(f'Spectral cut: {spectral_cut}')
print(f'Block cut: {block_cut}')
print(f'Random cut: {random_cut}')

Spectral cut: 10.440000000000001
Block cut: 10.440000000000001
Random cut: 45.893719806763286
